In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [2]:
df = pd.read_csv('../../../scrapy-hackathon/webscrape/linkedin/linkedin_dell.csv', usecols=['Post'])
df.head()

,Post
0,Proud to announce that my team won the recogni...
1,In SCDF and Dell's Innovation Lifesavers' Inno...
2,I am proud to announce that my TEAM TAGAR came...
3,As part of Dell Technologies' initiatives in s...
4,My internship at Dell Technologies has been a ...


In [3]:
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [4]:
def text_prep(x):
     corp = str(x).lower() 
     corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
     tokens = word_tokenize(corp)
     words = [t for t in tokens if t not in stop_words]
     lemmatize = [lemma.lemmatize(w) for w in words]
    
     return lemmatize

In [5]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ZY\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ZY\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
preprocess_tag = [text_prep(i) for i in df['Post']]
df["preprocess_txt"] = preprocess_tag

In [7]:
df['total_len'] = df['preprocess_txt'].map(lambda x: len(x))

In [8]:
file = open('negative-words.txt', 'r')
neg_words = file.read().split()
file = open('positive-words.txt', 'r')
pos_words = file.read().split()

In [9]:
num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
df['pos_count'] = num_pos
num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
df['neg_count'] = num_neg

In [10]:
#normalize the sentiments by dividing the result of +ve minus -ve by the total len of the post so that every sentiment of the post is standardized
df['sentiment'] = round((df['pos_count'] - df['neg_count']) / df['total_len'], 2)

In [15]:
posts = df['preprocess_txt']
posts

0      [proud, announce, team, recognition, people, c...
1      [scdf, dell, innovation, lifesaver, innovation...
2      [proud, announce, team, tagar, came, nd, place...
3      [part, dell, technology, initiative, shaping, ...
4      [internship, dell, technology, fun, enriching,...
                             ...                        
103    [bear, market, bear, market, able, tell, night...
104    [sliet, place, completed, engineering, one, ol...
105    [happy, announce, winning, team, second, year,...
106    [past, week, posted, anything, colleague, frie...
107    [entrepreneur, learn, next, young, tech, start...
Name: preprocess_txt, Length: 108, dtype: object

In [ ]:
#get top 10 most positive words
pos_dict = {}
for post in posts:
    for words in post:
        if words in pos_words:
            if pos_dict[words] in pos_dict:
                pos_dict[words] += 1
            else:
                pos_dict[words] = 1
df = pd.DataFrame(pos_dict, columns = ['Words', 'Word Count'])

In [11]:
df.to_csv('topten.csv')

In [12]:
df

,Post,preprocess_txt,total_len,pos_count,neg_count,sentiment
0,Proud to announce that my team won the recogni...,"[proud, announce, team, recognition, people, c...",15,1,0,0.07
1,In SCDF and Dell's Innovation Lifesavers' Inno...,"[scdf, dell, innovation, lifesaver, innovation...",183,22,4,0.10
2,I am proud to announce that my TEAM TAGAR came...,"[proud, announce, team, tagar, came, nd, place...",198,25,5,0.10
3,As part of Dell Technologies' initiatives in s...,"[part, dell, technology, initiative, shaping, ...",128,9,2,0.05
4,My internship at Dell Technologies has been a ...,"[internship, dell, technology, fun, enriching,...",43,8,1,0.16
...,...,...,...,...,...,...
103,Bear market? What bear market? You wouldn't be...,"[bear, market, bear, market, able, tell, night...",201,12,2,0.05
104,SLIET The place where I Completed My Engineeri...,"[sliet, place, completed, engineering, one, ol...",173,15,1,0.08
105,I`m so happy to announce Winning with my team ...,"[happy, announce, winning, team, second, year,...",24,5,0,0.21
106,"For the past few weeks, I have not posted anyt...","[past, week, posted, anything, colleague, frie...",148,12,2,0.07
